An LSF file is a binary file of concatenated IMC messages.
This notebook demonstrates some of the included functionality for parsing and exporting LSF files.

**Warning:** *Pyimc parses the LSF files under the assumption that the IMC specification is the same as the one it was compiled against.*

In [1]:
import pyimc
from pyimc.lsf import LSFReader, LSFExporter

lsf_path ='./Data.lsf'

## LSFReader usage

The LSFReader class can be instantiated using the 'with' statement to automatically handle file opening/closing.

In [2]:
# Print first three messages in LSF file
# save_index: a message index is generated, speeds up subsequent reads
with LSFReader(lsf_path, save_index=True) as lsf:
    for i, msg in zip(range(3), lsf.read_message()):
        # Print outputs the message type, destination/source imc address, timestamp and fields 
        print('Message', i)
        print(msg)
        print()

Message 0
LoggingControl
00D1:47 -> FFFF:FF
2020/02/11 08:35:21
    op: 1
    name: 20200211/083521_test_plan

Message 1
QueryEntityInfo
00D1:47 -> 00D1:00
2020/02/11 08:35:21
    id: 0

Message 2
QueryEntityInfo
00D1:47 -> 00D1:01
2020/02/11 08:35:21
    id: 1



The convenience function "read" returns a generator of messages of a certain type instead of using the "with" command.
Can be iterated over or converted to a list

In [3]:
msgs = list(LSFReader.read(lsf_path, types=[pyimc.EstimatedState]))
last_msg = msgs[-1]
print(last_msg)

EstimatedState
00D1:28 -> FFFF:FF
2020/02/11 08:36:40
    lat: 0.718814
    lon: -0.151952
    height: -0.168725
    x: -147.467
    y: 72.6613
    z: 0.285875
    phi: -0.000253006
    theta: 0.235803
    psi: -1.3983
    u: 1.08701
    v: 0.000631064
    w: 0.0149416
    vx: 0.182613
    vy: -1.04441
    vz: -0.239383
    p: -0.00140179
    q: 5.05482e-05
    r: 0.000335344
    depth: 0.0805944
    alt: 6.02733


## LSFExporter usage

This class reads LSF files and exports it to a Pandas DataFrame.

In [4]:
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', 30)

exp = LSFExporter(lsf_path)
df = exp.export_messages(imc_type=pyimc.EstimatedState)

# Print first five rows
display(df.head())

,timestamp,src,src_ent,dst,dst_ent,lat,lon,height,phi,theta,psi,u,v,w,vx,vy,vz,p,q,r,depth,alt
0,2020-02-11 08:35:21.745552778,lauv-simulator-1,Navigation,*,*,0.718787,-0.151942,-1.104833,-0.000796,0.001342,1.578148,0.020301,0.003116,0.037576,-0.003296,0.020382,0.037546,-0.000144,-0.000739,-0.000359,0.083131,6.443270
1,2020-02-11 08:35:21.795579910,lauv-simulator-1,Navigation,*,*,0.718787,-0.151942,-1.080496,0.000111,0.001479,1.577950,0.020172,0.001933,0.023263,-0.003259,0.020312,0.023233,-0.000643,-0.000528,0.000573,0.107469,6.443270
2,2020-02-11 08:35:21.845555544,lauv-simulator-1,Navigation,*,*,0.718787,-0.151942,-1.080496,0.000090,0.000866,1.578091,0.020043,0.001933,0.023263,-0.002078,0.020177,0.023246,0.000641,0.000313,0.000331,0.107469,6.419576
3,2020-02-11 08:35:21.895576000,lauv-simulator-1,Navigation,*,*,0.718787,-0.151942,-1.109738,-0.000421,0.000247,1.578182,0.019652,0.001933,0.006528,-0.002084,0.020029,0.006522,-0.000226,0.000021,-0.000395,0.078227,6.419576
4,2020-02-11 08:35:21.945562601,lauv-simulator-1,Navigation,*,*,0.718787,-0.151942,-1.109738,0.000129,0.001114,1.578424,0.019261,0.001933,0.006528,-0.002082,0.019644,0.006506,0.000097,0.000007,0.000439,0.078227,6.419576


In [5]:
import numpy as np

# Index by timestamp instead
df_t = df.set_index('timestamp')

# Get all rows between two timestamps
t_start = np.datetime64('2020-02-11 08:36:00')
t_end = np.datetime64('2020-02-11 08:36:10')
df_slice = df_t[t_start:t_end]
display(df_slice)

,src,src_ent,dst,dst_ent,lat,lon,height,phi,theta,psi,u,v,w,vx,vy,vz,p,q,r,depth,alt
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-02-11 08:36:00.045565128,lauv-simulator-1,Navigation,*,*,0.718786,-0.151933,-1.178009,0.002779,0.250984,0.389274,1.284798,-0.027727,0.009806,1.164706,0.447772,-0.309805,-0.011366,0.002767,0.009514,0.024308,5.960278
2020-02-11 08:36:00.095555782,lauv-simulator-1,Navigation,*,*,0.718786,-0.151933,-1.199915,0.002332,0.251501,0.389579,1.284762,-0.027727,-0.021868,1.156599,0.444938,-0.340974,-0.010631,0.000840,0.007332,0.002210,5.960278
2020-02-11 08:36:00.145553112,lauv-simulator-1,Navigation,*,*,0.718786,-0.151933,-1.199723,0.002261,0.250874,0.389635,1.284725,-0.027727,-0.021868,1.156739,0.445070,-0.340187,-0.008828,0.002070,0.005509,0.002210,5.960278
2020-02-11 08:36:00.195568323,lauv-simulator-1,Navigation,*,*,0.718786,-0.151933,-1.141709,0.001365,0.251228,0.389930,1.284311,-0.020780,-0.007465,1.159796,0.446679,-0.326641,-0.007769,0.002206,0.003686,0.060034,5.960278
2020-02-11 08:36:00.245560646,lauv-simulator-1,Navigation,*,*,0.718786,-0.151933,-1.141518,0.000971,0.251182,0.389899,1.283896,-0.020780,-0.007465,1.156817,0.452920,-0.326464,-0.008077,0.002041,-0.000789,0.060034,5.959414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-11 08:36:09.795555353,lauv-simulator-1,Navigation,*,*,0.718788,-0.151932,-1.104022,-0.000899,0.248698,0.268806,1.211239,0.001795,-0.010151,1.128662,0.314313,-0.308011,0.001397,-0.000258,-0.002074,0.019547,6.461446
2020-02-11 08:36:09.845554590,lauv-simulator-1,Navigation,*,*,0.718788,-0.151932,-1.103835,-0.000488,0.248817,0.268874,1.211104,0.001795,-0.010151,1.128874,0.312913,-0.308115,0.002533,-0.000770,-0.001551,0.019547,6.461684
2020-02-11 08:36:09.895563364,lauv-simulator-1,Navigation,*,*,0.718788,-0.151932,-1.099301,-0.000167,0.247988,0.268865,1.211262,0.001795,-0.001893,1.130950,0.313479,-0.299105,0.002464,-0.000592,-0.000201,0.023894,6.461684


In [6]:
# Output data to a csv file
df_slice.to_csv('output.csv')

In [7]:
# Read another message type using the same exporter (re-use metadata)
df2 = exp.export_messages(imc_type=pyimc.Announce)

# Remove all messages not originating from lauv-simulator-1
df2 = df2[df2['src'] == 'lauv-simulator-1']
display(df2.head(2))

# Alternatively, supply a lambda predicate to the export_messages function (avoids converting unnecessary messages to pandas)
imc_id = exp.get_node_id('lauv-simulator-1')
df3 = exp.export_messages(imc_type=pyimc.Announce, condition=lambda msg: msg.src == imc_id)
display(df3.head(2))

,timestamp,src,src_ent,dst,dst_ent,sys_name,sys_type,owner,lat,lon,height,services
2,2020-02-11 08:35:31.412159920,lauv-simulator-1,Service Announcer,0x0,*,lauv-simulator-1,UUV,65535,0.718787,-0.151941,-1.173098,dune://0.0.0.0/uid/74719034579618;dune://0.0.0...
3,2020-02-11 08:35:31.412170410,lauv-simulator-1,Service Announcer,0x0,*,lauv-simulator-1,UUV,65535,0.718787,-0.151941,-1.173098,dune://0.0.0.0/uid/74719034579618;dune://0.0.0...


,timestamp,src,src_ent,dst,dst_ent,sys_name,sys_type,owner,lat,lon,height,services
0,2020-02-11 08:35:31.412159920,lauv-simulator-1,Service Announcer,0x0,*,lauv-simulator-1,UUV,65535,0.718787,-0.151941,-1.173098,dune://0.0.0.0/uid/74719034579618;dune://0.0.0...
1,2020-02-11 08:35:31.412170410,lauv-simulator-1,Service Announcer,0x0,*,lauv-simulator-1,UUV,65535,0.718787,-0.151941,-1.173098,dune://0.0.0.0/uid/74719034579618;dune://0.0.0...
